In [1]:
import pandas as pd
from tqdm.notebook import tqdm 
import gc
import numpy as np

In [2]:
import statistics as stat
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [3]:
path = '../data'
y_train = pd.read_csv(f'{path}/y_train.csv', index_col = 'key_value')

## Procesar RCC

In [ ]:
rcc_train = pd.read_csv(f'{path}/rcc_train.csv')
rcc_test = pd.read_csv(f'{path}/rcc_test.csv')

In [ ]:
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [ ]:
dict_ = {'codmes': 'int32',
 'key_value': 'int32',
 'condicion': 'int32',
 'tipo_credito': 'int32',
 'cod_instit_financiera': 'int32',
 'PRODUCTO': 'int32',
 'RIESGO_DIRECTO': 'int32',
 'COD_CLASIFICACION_DEUDOR': 'int32'}
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [ ]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins)
rcc_train["condicion"] = rcc_train["condicion"].cat.codes
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins)
rcc_test["condicion"] = rcc_test["condicion"].cat.codes

In [ ]:
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'cod_instit_financiera':['nunique','min','max',moda],
           'PRODUCTO':['nunique','min','max',moda],}

In [ ]:
list_rcc_train_agg = []
for n,i in enumerate(sorted(set(rcc_train.codmes),reverse=True)):
    for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
        print(f'haciendo {c} desde {i}')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoSum_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.size().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoUnique_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.min().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMin_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.max().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMax_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.std().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoStd_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
    print(f'haciendo aggregate de {i}')
    rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
    list_rcc_train_agg.append(rcc_train_agg)
    gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

In [ ]:
list_rcc_test_agg = []
for n,i in enumerate(sorted(set(rcc_test.codmes),reverse=True)):
    for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
        print(f'haciendo {c} desde {i}')
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoSum_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.size().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoUnique_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.min().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMin_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.max().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMax_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.std().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoStd_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

    print(f'haciendo aggregate de {i}')
    rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
    list_rcc_test_agg.append(rcc_test_agg)
    gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

In [ ]:
### asegurar que las columnas esten en ambas bases (train/test)
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

In [ ]:
### unir RCC en la base final
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

## Procesar SE

In [ ]:
se_train = pd.read_csv(f'{path}/se_train.csv', index_col = 'key_value')
se_test = pd.read_csv(f'{path}/se_test.csv', index_col = 'key_value')

In [ ]:
dict_ = {'sexo':'int32',
         'est_cvl':'int32',
         'sit_lab':'int32',
         'cod_ocu':'int32',
         'ctd_hijos':'int32',
         'flg_sin_email':'int32',
         'ctd_veh':'int32',
         'lgr_vot':'int32',
         'prv':'int32',
         'dto':'int32',
         'rgn':'int32',
         'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [ ]:
### unir SE en la base final
train = train.join(se_train) 
test = test.join(se_test)
del se_train, se_test

## Procesar SUNAT

In [ ]:
sunat_train = pd.read_csv(f'{path}/sunat_train.csv')
sunat_test = pd.read_csv(f'{path}/sunat_test.csv')

In [ ]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)
sunat_train.shape, sunat_test.shape

In [ ]:
dict_ = {'tipcontribuyente': 'int32',
         'tippersona': 'int32',
         'ciiu': 'int32',
         'ubigeo': 'int32',
         'condiciondomicilio': 'int32',
         'estadocontribuyente': 'int32',
         'codvia': 'int32',
         'codzona': 'int32',
         'contabilidad': 'int32',
         'facturacion': 'int32',
         'domiciliado': 'int32',
         'comercioexterior': 'int32',
         'cargorele': 'int32',
         'codentidadtributo': 'int32',
         'estadotributo': 'int32'}
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [ ]:
sunat_train['diff_fech'] = sunat_train['fecbaja'] - sunat_train['fecalta']
sunat_test['diff_fech'] = sunat_test['fecbaja'] - sunat_test['fecalta']

In [ ]:
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['mean','max', 'nunique','min','std'],
           'fecbaja':['mean','max', 'nunique','min','std'], 
           'diff_fech':['mean','max', 'nunique','min','std'], 
            }

sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [ ]:
### unir SUNAT en la base final
train = train.join(sunat_train_)
test = test.join(sunat_test_)
del sunat_train_, sunat_test_

## Training
* despues de realizar varias iteraciones empleando el modelo lightGBM para identificar las variables. Eliminado aquellas que tienen zero_importance y low_importance (cuando su valor de importancia no contribuye con el 0.99 de importancia acumulativo). Parar hasta que no exista variable con zero_importancia

In [4]:
# # eliminar variables con zero_importance
# zero_importance = fi[fi==0]
# aux = fi[fi>0].sort_values(ascending=False)
# keep_columns = []
# count = 0
# for feature,values in zip(aux.index, aux.values):
#     count+=values
#     if count<=0.99:
#         keep_columns.append(feature)
        
# len(keep_columns), len(zero_importance), len(train.columns)

In [5]:
# train = pd.read_pickle('train_1799features.pkl')
# test = pd.read_pickle('test_1799features.pkl')
# #### guardar archivo procesado
# train=pd.read_pickle('train_907features.pkl')
# test = pd.read_pickle('test_907features.pkl')
# ## guardar archivo procesado
train = pd.read_pickle('train_886features.pkl')
test = pd.read_pickle('test_886features.pkl')

In [6]:
train.shape, test.shape

((358487, 886), (396666, 886))

In [7]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index].target

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index].target

    learner = LGBMClassifier(n_estimators=1000, boosting_type='gbdt',min_child_samples=1500, 
                   colsample_bytree=0.8,subsample=0.8, max_bin=200, learning_rate=0.1)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))  

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.844244	training's binary_logloss: 0.30249	valid_1's auc: 0.836231	valid_1's binary_logloss: 0.308795
[100]	training's auc: 0.858563	training's binary_logloss: 0.291167	valid_1's auc: 0.845098	valid_1's binary_logloss: 0.301731
[150]	training's auc: 0.866926	training's binary_logloss: 0.284583	valid_1's auc: 0.848041	valid_1's binary_logloss: 0.299274
[200]	training's auc: 0.873269	training's binary_logloss: 0.279527	valid_1's auc: 0.849593	valid_1's binary_logloss: 0.297966
[250]	training's auc: 0.878481	training's binary_logloss: 0.275355	valid_1's auc: 0.850331	valid_1's binary_logloss: 0.297354
[300]	training's auc: 0.883379	training's binary_logloss: 0.271322	valid_1's auc: 0.85079	valid_1's binary_logloss: 0.29691
Early stopping, best iteration is:
[311]	training's auc: 0.884357	training's binary_logloss: 0.270534	valid_1's auc: 0.850842	valid_1's binary_logloss: 0.296852
**

In [8]:
fi.sort_values(ascending=False).head(50)

RIESGO_DIRECTO_1_saldoUnique_ult1meses               0.013812
edad                                                 0.013588
cod_instit_financiera_max_ult1mes                    0.011832
tip_lvledu                                           0.010493
COD_CLASIFICACION_DEUDOR_0_saldoUnique_ult12meses    0.010095
RIESGO_DIRECTO_-1_saldoUnique_ult12meses             0.010030
RIESGO_DIRECTO_1_saldoMin_ult1meses                  0.009826
cod_ubi                                              0.009664
RIESGO_DIRECTO_-1_saldoSum_ult1meses                 0.009544
RIESGO_DIRECTO_1_saldoStd_ult1meses                  0.009045
prv                                                  0.007892
sit_lab                                              0.007671
cod_instit_financiera_min_ult1mes                    0.007635
RIESGO_DIRECTO_1_saldoMax_ult12meses                 0.007565
ubigeo_mode                                          0.007520
cod_instit_financiera_mode_ult1mes                   0.007411
tipo_cre

In [9]:
test_probs.name='target'
test_probs.to_csv('../results/test_LGBM_886Features_0.8502134.csv') ### score de 0.84573 en la tabla publica 